# Experiment with the horse or human classifier


Run the code below to download the dataset horse-or-human.zip.

In [2]:
!pip install gdown
!gdown --id 1onaG42NZft3wCE1WH0GDEbUhu75fedP5

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Created wheel for gdown: filename=gdown-4.0.2-py3-none-any.whl size=10598 sha256=763078b7a3d7fd572fbe50c5082b84d88bb956a3bb3624926fd62de5f734da5a
  Stored in directory: /home/indrap24/.cache/pip/wheels/65/8a/7d/7471d53c4c8ac472998ccd01a56de910be3fc232c8db530c4c
Successfully built gdown
Downloading...
From: https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5
To: /mnt/569013e9-d6c9-40ca-b4a9-2355022e7589/Projects/TensorFlow/TensorFlow-Tutorial/DeepLearning.AI TensorFlow Developer/C4_Using Real World Images.ipynb/horse-or-human.zip
100%|█████████████████████████████████████████| 150M/150M [04:39<00:00, 536kB/s]


In [10]:
!unzip './horse-or-human.zip' -d './horse-or-human' && rm './horse-or-human.zip'

Archive:  ./horse-or-human.zip
  inflating: ./horse-or-human/horses/horse01-0.png  
  inflating: ./horse-or-human/horses/horse01-1.png  
  inflating: ./horse-or-human/horses/horse01-2.png  
  inflating: ./horse-or-human/horses/horse01-3.png  
  inflating: ./horse-or-human/horses/horse01-4.png  
  inflating: ./horse-or-human/horses/horse01-5.png  
  inflating: ./horse-or-human/horses/horse01-6.png  
  inflating: ./horse-or-human/horses/horse01-7.png  
  inflating: ./horse-or-human/horses/horse01-8.png  
  inflating: ./horse-or-human/horses/horse01-9.png  
  inflating: ./horse-or-human/horses/horse02-0.png  
  inflating: ./horse-or-human/horses/horse02-1.png  
  inflating: ./horse-or-human/horses/horse02-2.png  
  inflating: ./horse-or-human/horses/horse02-3.png  
  inflating: ./horse-or-human/horses/horse02-4.png  
  inflating: ./horse-or-human/horses/horse02-5.png  
  inflating: ./horse-or-human/horses/horse02-6.png  
  inflating: ./horse-or-human/horses/horse02-7.png  
  inflating: ./

The contents of the .zip are extracted to the base directory `./horse-or-human`, which in turn each contain horses and humans subdirectories.

In short: The training set is the data that is used to tell the neural network model that 'this is what a horse looks like', 'this is what a human looks like' etc.

One thing to pay attention to in this sample: We do not explicitly label the images as horses or humans. If you remember with the handwriting example earlier, we had labelled 'this is a 1', 'this is a 7' etc. Later you'll see something called an ImageGenerator being used -- and this is coded to read images from subdirectories, and automatically label them from the name of that subdirectory. So, for example, you will have a 'training' directory containing a 'horses' directory and a 'humans' one. ImageGenerator will label the images appropriately for you, reducing a coding step.

Let's define each of these directories:

In [ ]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')